# Captain Cook: the fabulous recipes explorator



Objectives:

- Create our own JSON map to plot informations about the recipes by region
- Finish the ingredients list cleaning
- Use statistical properties of the English language
- Create a user friendly recipe finder 


Bonus:

- Try to compute missing nutritional informations
- Find meaningful substitutions for ingredients

In [55]:
# Basic imports
import re
import os.path
import numpy as np
import scipy as sp
import pandas as pd

# Map-related imports
import json
import branca
import folium
from pandas.io.json import json_normalize
from IPython.core.display import display, HTML

# Plot-related imports
import seaborn as sns
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

# NLP-related imports
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/kumilla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kumilla/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# General parameters
%matplotlib inline
plt.style.use('seaborn')#switch to seaborn style
plt.rcParams["figure.figsize"] = [16,10]

DATA_FOLDER = './data/'

# 1. Data Loading
  
The Data has been fetched and cleaned with `BASH`scripts, please look in the *dataCleaning* section to understand how this was achieved.

**Home made fetched dataset:**

In [4]:
# Importing ingredients to Pandas DF
allrecipes_df = pd.read_csv(DATA_FOLDER + 'allrecipes.csv', sep='\t',  header=None, encoding = "utf-8")
allrecipes_df.columns = ['ID', 'Region', 'Title', 'Ingredients', 'kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol']

# Bug?? need to convert into numeric somes, TODO EFFICIENT WAY TO DO THIS???
allrecipes_df['kcal'] = pd.to_numeric(allrecipes_df['kcal'], errors='coerce')
allrecipes_df['carb'] = pd.to_numeric(allrecipes_df['carb'], errors='coerce') / 1000.0 # convert to g
allrecipes_df['fat'] = pd.to_numeric(allrecipes_df['fat'], errors='coerce') / 1000.0 # convert to g
allrecipes_df['protein'] = pd.to_numeric(allrecipes_df['protein'], errors='coerce')
allrecipes_df['sodium'] = pd.to_numeric(allrecipes_df['sodium'], errors='coerce') / 1000.0
allrecipes_df['cholesterol'] = pd.to_numeric(allrecipes_df['cholesterol'], errors='coerce')

# Remove any rows which isn't properly formatted
allrecipes_df = allrecipes_df.dropna()

# Remove any duplicated lines
allrecipes_df = allrecipes_df.drop_duplicates().set_index('ID')

# Printing
allrecipes_df.head(5)

,Region,Title,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,,
b9705d990df6857f20756fc996a54b63,us,Traditional Indiana Persimmon Pudding,2 cups persimmon pulp |2 eggs |1 cup white sug...,278.0,53.9,3.8,7.8,0.224,35.0
4658708d644d7b446d843fed5ddf60c4,us,Fish Tacos,1 cup all-purpose flour |2 tablespoons cornsta...,409.0,43.0,18.8,17.3,0.407,54.0
beed004e2a1772ba0db9da913f54122e,us,Wisconsin Slow Cooker Brats,8 bratwurst |2 (12 fluid ounce) cans or bottle...,377.0,12.8,27.4,13.8,1.046,69.0
96353c72421bd74096277c6cf8b17097,us,Buffalo Chicken Wing Sauce,2/3 cup hot pepper sauce (such as Frank&#39;s ...,104.0,0.4,11.6,0.2,0.576,31.0
ee659a6a5e69834b60744cc3e103729e,us,Minnesota's Favorite Cookie,"1 cup butter, softened |1 1/2 cups brown sugar...",140.0,14.9,8.7,1.5,0.076,22.0


In [5]:
# Importing descriptions to Pandas DF
allrecipes_desc_df = pd.read_csv(DATA_FOLDER + 'allrecipes_desc.csv', sep='£',  header=None, encoding = "utf-8",  engine='python')
allrecipes_desc_df.columns = ['ID', 'Description']

# Remove any duplicated lines
allrecipes_desc_df = allrecipes_desc_df.drop_duplicates().set_index('ID')

allrecipes_desc_df.head(5)

,Description
ID,
b9705d990df6857f20756fc996a54b63,Preheat the oven to 350 degrees F (175 degree...
4658708d644d7b446d843fed5ddf60c4,"To make beer batter: In a large bowl, combine..."
beed004e2a1772ba0db9da913f54122e,"Place bratwurst, beer, onion, and ketchup in ..."
96353c72421bd74096277c6cf8b17097,"Combine the hot sauce, butter, vinegar, Worce..."
ee659a6a5e69834b60744cc3e103729e,Preheat oven to 350 degrees F (175 degrees C)...


In [6]:
print("Number of recipes:", len(allrecipes_df.index.unique()))

Number of recipes: 15894


**Provided Dataset**

This dataset was provided with the assignment and cleaned with the provided `Perl` scripts. 

Thanks to the scripts, we obtain two datasets:

1. `cleaned_ing.csv` contains the list of ingredients for each recipe,
2. `cleaned_nutri.csv` contains the corresponding nutritional values.

Our objective is to merge these two sets to obtain a unique set with all useful informations.

In [30]:
# Importing ingredients to Pandas DF
ing_df = pd.read_csv(DATA_FOLDER + 'cleaned_ing.csv', sep='\t',  header=None, encoding = "utf-8")
ing_df.columns = ['ID', 'Title', 'Ingredients']

# Importing nutritional values to Pandas DF
nutri_df = pd.read_csv(DATA_FOLDER + 'cleaned_nutri.csv', sep='\t',  header=None, encoding = "utf-8")
nutri_df.columns = ['ID', 'kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol']

# Merging
ing_df = ing_df.set_index('ID')
nutri_df = nutri_df.set_index('ID')
provided_df = ing_df.merge(nutri_df, on='ID', how='inner')

# Drop NaNs and duplicate lines
provided_df = provided_df.dropna().drop_duplicates()

provided_df.head()

,Title,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,
38e1b80017526d6e59ed3f986c35a43a,T.G.I. Friday's Jack Daniels Sauce Recipe #10265,1 teaspoon onion powder|1 tablespoon Tabasco s...,?,?,?,?,?,?
a3636a4dab434fe21fbcdceba7d6fcf2,Simple Peanut Squash Recipe,1 butternut squash|2 tablespoons brown sugar|1...,536,86.4,23.6,6.5,483,61
117f3c214e9de550a157ce5ee1f1cceb,Hash Brown Breakfast Casserole Recipe,"1 lb ground sausage (""hot"" or ""sage"" flavored)...",660.4,24.7,47.3,32.5,1248.0,251.6
fde8f280a690fb8bc77c10a7193db08b,Basic Homemade Country Sausage Recipe,2 pounds lean pork|1/2 pound pork fatback|3 te...,?,?,?,?,?,?
714df642f50b9ae489d285e16b59bf7b,Spinach Frittata Recipe,1 cup fresh spinach|2 egg whites|1 egg yolk|1/...,?,?,?,?,?,?


We can observe that some nutritional values are missing, which can be solved either by removing the lines or by trying to calculate these values from the given ingredients.

As trying to calculate the values from ingredients with different units (i.e. grams, cups, tbsp, etc) requires a set of informations that we do not have, we decided to leave these lines as they are for now. 

In [31]:
# Bug?? need to convert into numeric somes, TODO EFFICIENT WAY TO DO THIS???
provided_df['kcal'] = pd.to_numeric(provided_df['kcal'], errors='coerce')
provided_df['carb'] = pd.to_numeric(provided_df['carb'], errors='coerce')
provided_df['fat'] = pd.to_numeric(provided_df['fat'], errors='coerce')
provided_df['protein'] = pd.to_numeric(provided_df['protein'], errors='coerce')
provided_df['sodium'] = pd.to_numeric(provided_df['sodium'], errors='coerce')
provided_df['cholesterol'] = pd.to_numeric(provided_df['cholesterol'], errors='coerce')

# Insert Region column to match the other DF
provided_df.insert(loc=1, column='Region', value=np.nan)
provided_df.head(5)

,Title,Region,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,,
38e1b80017526d6e59ed3f986c35a43a,T.G.I. Friday's Jack Daniels Sauce Recipe #10265,NaN,1 teaspoon onion powder|1 tablespoon Tabasco s...,NaN,NaN,NaN,NaN,NaN,NaN
a3636a4dab434fe21fbcdceba7d6fcf2,Simple Peanut Squash Recipe,NaN,1 butternut squash|2 tablespoons brown sugar|1...,536.0,86.4,23.6,6.5,483.0,61.0
117f3c214e9de550a157ce5ee1f1cceb,Hash Brown Breakfast Casserole Recipe,NaN,"1 lb ground sausage (""hot"" or ""sage"" flavored)...",660.4,24.7,47.3,32.5,1248.0,251.6
fde8f280a690fb8bc77c10a7193db08b,Basic Homemade Country Sausage Recipe,NaN,2 pounds lean pork|1/2 pound pork fatback|3 te...,NaN,NaN,NaN,NaN,NaN,NaN
714df642f50b9ae489d285e16b59bf7b,Spinach Frittata Recipe,NaN,1 cup fresh spinach|2 egg whites|1 egg yolk|1/...,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
print("Number of recipes:", len(provided_df.index.unique()))

Number of recipes: 31376


In [20]:
# Concatenate the 2 DF and drop any duplicated lines, it is possible since some data come from the same website!
recipes_df = allrecipes_df.append(provided_df, sort=False).drop_duplicates()
recipes_df['Region'] = recipes_df['Region'].astype('category')

recipes_df.head()

,Region,Title,Ingredients,kcal,carb,fat,protein,sodium,cholesterol
ID,,,,,,,,,
b9705d990df6857f20756fc996a54b63,us,Traditional Indiana Persimmon Pudding,2 cups persimmon pulp |2 eggs |1 cup white sug...,278.0,53.9,3.8,7.8,0.224,35.0
4658708d644d7b446d843fed5ddf60c4,us,Fish Tacos,1 cup all-purpose flour |2 tablespoons cornsta...,409.0,43.0,18.8,17.3,0.407,54.0
beed004e2a1772ba0db9da913f54122e,us,Wisconsin Slow Cooker Brats,8 bratwurst |2 (12 fluid ounce) cans or bottle...,377.0,12.8,27.4,13.8,1.046,69.0
96353c72421bd74096277c6cf8b17097,us,Buffalo Chicken Wing Sauce,2/3 cup hot pepper sauce (such as Frank&#39;s ...,104.0,0.4,11.6,0.2,0.576,31.0
ee659a6a5e69834b60744cc3e103729e,us,Minnesota's Favorite Cookie,"1 cup butter, softened |1 1/2 cups brown sugar...",140.0,14.9,8.7,1.5,0.076,22.0


In [28]:
print("Number of total recipes:", len(recipes_df.index.unique()))

Number of total recipes: 46999


In [29]:
len(recipes_df[recipes_df['Region']=='italian'])/365

6.841095890410959

We see that the total number of recipes is enough to eat italian recipes everyday for almost 7 years!!

# 2. Analysis

This part presents some basic statistical analysis of the data.

First we analyse the data by region and observe *mean*, *median*, *min* and *max* for each nutritional value.

In [33]:
# Some classic analysis
recipes_regions = recipes_df.groupby('Region')
recipes_regions = recipes_regions.agg({'kcal' : ['mean', 'median', 'min', 'max'],
                                       'carb' : ['mean', 'median', 'min', 'max'],
                                       'fat' : ['mean', 'median', 'min', 'max'],
                                       'protein' : ['mean', 'median', 'min', 'max'],
                                       'sodium' : ['mean', 'median', 'min', 'max'],
                                       'cholesterol' : ['mean', 'median', 'min', 'max']})
recipes_regions.sort_values([('kcal', 'mean')], ascending=False).head()

kcal                            carb                     \
                  mean median   min     max       mean median  min    max   
Region                                                                      
malaysian   435.600000  427.0  33.0  1238.0  30.422857  26.70  1.7  113.2   
portuguese  402.173333  378.0  27.0  2266.0  33.832000  28.50  4.0  108.6   
italian     391.215459  360.0   4.0  1641.0  34.242171  29.90  0.0  174.8   
indonesian  387.648649  401.0  65.0   716.0  29.056757  19.60  6.5   94.2   
french      377.932609  319.5   9.0  3274.0  27.991739  22.05  0.4  240.9   

                  fat         ...   protein           sodium                 \
                 mean median  ...       min    max      mean  median    min   
Region                        ...                                             
malaysian   24.091429  19.80  ...       0.9   85.2  0.528771  0.3440  0.011   
portuguese  20.009333  17.10  ...       0.2  101.0  1.186107  0.4990  0.003   
italian     19.998438  17.20  ...       0.0   83.3  0.704962  0.5680  0.001   
indonesian  21.545946  19.00  ...       0.0   47.6  0.745324  0.4910  0.004   
french      22.572391  18.15  ...       0.0   76.7  0.496848  0.3005  0.002   

                   cholesterol                     
               max        mean median  min    max  
Region                                             
malaysian    3.031   90.914286   66.0  0.0  340.0  
portuguese  10.693   84.986667   67.0  0.0  389.0  
italian      7.648   71.113736   50.0  0.0  734.0  
indonesian   2.459   97.918919   68.0  0.0  500.0  
french       8.623  100.473913   73.5  0.0  780.0  

[5 rows x 24 columns]

# 3. Ingredient Parsing

In this part we are trying to get a list of ingredients for each recipe. This list should be clean, which means it should contain only the names of the ingredients and no other informations, like quantities.

To do this, first we cleaned the list of ingredients by applying a low-case and by removing a set of words chosen manually (contained in `black_list`), then we used the natural language processing library `nltk` to remove words different from nouns.

In [54]:
# Copy for test
recipes_copy = recipes_df.copy()

# lowercase to be insensitive
recipes_copy['Ingredients'] = recipes_copy['Ingredients'].str.lower()

# Coerce filtering, removing any occurence of these words as a first filter
black_list = ['inches','inch','medium','pounds','pound','ounces','ounces','fluid','ground','tablespoons','tablespoon','cups','cup','teaspoons','teaspoon', 'all-purpose', '\(.*\)']
recipes_copy['Ingredients'] = recipes_copy['Ingredients'].replace(black_list, '', regex=True)

# Remove non alphabetic values expect of '|' which is the seperating char
recipes_copy['Ingredients'] = recipes_copy['Ingredients'].str.replace('[^a-zA-Z ]+', ' ')

# Retrieve list of ingredients in overall
keywords_list = ingredient_serie.str.split(" ", expand=True).stack().unique()

array(['', 'persimmon', 'pulp', ..., 'oolong', 'stage', 'hmr'],
      dtype=object)

In [56]:
### Retrieve bad ingredients

# NLP to identify only verbs
tokens = nltk.word_tokenize(' '.join(keywords_list))
tagged = nltk.pos_tag(tokens)

# Fetching the list of non correct word
gray_list = [word for word,pos in tagged if not(pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

# Further filtering by removing gray_listed word with regex
ingredient_serie = ingredient_serie.replace(gray_list, '')

# Retrieve list of ingredients in overall
keywords_list = ingredient_serie.str.split(" ", expand=True).stack().unique()

In [57]:
# NLP to identify only nouns
tokens = nltk.word_tokenize(' '.join(keywords_list))
tagged = nltk.pos_tag(tokens)
nouns = [word for word,pos in tagged if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]

# We need to remove word that a smaller than 3 letters, as we suppose they are not ingredients
ing_list = [item for item in nouns if len(item) > 3]

At this point, we have a list of ingredients contained in `ing_list`, which can be used to filter our dataset. Unfortunately, as we are going to see below, some ingredients are not spelled correctly while others are not ingredients at all.

In [60]:
# Take original ingredients list and split each word to count recurrencies
ing_ds = recipes_copy['Ingredients'].str.split(" ", expand=True) \
                                        .stack().value_counts()  \
                                        .to_frame(name='count')  \
                                        .reset_index()

# Keeping only the ingredient in the previous list
ing_ds = ing_ds[ing_ds['index'].isin(ing_list)].reset_index(drop=True)

#ing_ds.sort_values(by='index') # if you want to see similar words
ing_ds.head(21)

,index,count
0,salt,22841
1,pepper,20391
2,butter,13037
3,onion,11935
4,flour,11890
5,taste,10342
6,water,10221
7,powder,9551
8,milk,7421
9,sauce,7414


We can see that the words `powder`, `taste` and `sauce` are contained in the ten most recurring words, although they are not ingredients. These words should then be parsed by hand and removed to obtain a list that is ingredients-only.

We can also notice that some ingredients are duplicated due to different spellings (i.e. `onion` and `onions`). 

We tried to implement a way to merge similar words by finding a metric that calculates the distance between words. As such the similar words should have close values given by the metric.

In [ ]:
### Retrieve similar names

# We can create a space with N dimensions
# Each letter of a word is mapped to its corresponding integer in this space
# Similar words will lie closely in this space

# Convert ingredient's distribution to list  
ing_ds_list = ing_ds['index'].values.tolist()
# print("\033[1mbar before sort:\033[0m", ing_ds_list)

# Looking for the longest word/ingredient
N = len(sorted(ing_ds_list, key=len)[-1])
# print("\033[1m\nLongest word is:\033[0m", N, " long")

# For each word in the list, we append the NULL element ASCII to have the same number of elements
converted_ing_list = [item + chr(0) * (N - len(item)) for item in ing_ds_list]
# print("\033[1m\n converted_ing_list after padding:\033[0m\n", converted_ing_list)

# Convert into each spatiales ASCII -> Numpy matrix
word_matrix = np.array([[ord(char) for char in string] for string in converted_ing_list])
# print("\033[1m\n converted_ing_list after ASCII int conversion:\033[0m\n", word_matrix)

In [65]:
# Compute the distance between each row 
# Idea: use backwards propagation to calculate the optimal weights
w = [10, 4, 3, 2, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 , 1, 1]
distance_matrix = sp.spatial.distance.cdist(word_matrix, word_matrix, 'wminkowski', p=2, w=w)
#distance_matrix = sp.spatial.distance.cdist(word_matrix, word_matrix, 'euclidean')

# print("\033[1m\nDistance of the matrix define by converted_ing_list:\033[0m\n", distance_matrix)

In [66]:
# Thresholding <-> if the distance is small enough words are the same!
normed_dist = (distance_matrix < 60).astype(int)
# print("\033[1m\nDistance of the matrix thresholded:\033[0m\n", normed_dist)

# The list has been sorted
# if we take the first non-zero value for each row we get the matching word
vec = normed_dist.argmax(axis=0)
# print("\033[1m\nIndex of corresponding words in sorted [converted_ing_list]:\033[0m\n", vec)

# Foo after TODO name
deconverted_ing_list = [converted_ing_list[i].replace(chr(0), '') for i in vec]
# print("\033[1m deconverted_ing_list operation:\033[0m", deconverted_ing_list)

In [67]:
# Result
ing_dict = dict(zip(ing_ds_list, deconverted_ing_list))

ing_dict

{'salt': 'salt',
 'pepper': 'pepper',
 'butter': 'butter',
 'onion': 'onion',
 'flour': 'flour',
 'taste': 'taste',
 'water': 'taste',
 'powder': 'pepper',
 'milk': 'milk',
 'sauce': 'taste',
 'chicken': 'chicken',
 'cloves': 'butter',
 'cream': 'cream',
 'eggs': 'eggs',
 'juice': 'juice',
 'vanilla': 'vanilla',
 'lemon': 'onion',
 'vinegar': 'vanilla',
 'beef': 'eggs',
 'cinnamon': 'cinnamon',
 'onions': 'pepper',
 'tomatoes': 'tomatoes',
 'bell': 'eggs',
 'parsley': 'parsley',
 'slices': 'pepper',
 'pieces': 'pepper',
 'boneless': 'cinnamon',
 'rice': 'salt',
 'tomato': 'powder',
 'bread': 'cream',
 'potatoes': 'tomatoes',
 'celery': 'cloves',
 'basil': 'basil',
 'cilantro': 'cinnamon',
 'cumin': 'flour',
 'frozen': 'butter',
 'chocolate': 'chocolate',
 'peppers': 'parsley',
 'skinless': 'tomatoes',
 'clove': 'flour',
 'margarine': 'margarine',
 'lime': 'milk',
 'cheddar': 'chicken',
 'corn': 'eggs',
 'breast': 'cloves',
 'orange': 'onions',
 'sweet': 'onion',
 'pork': 'milk',
 'carr

As we can see, this method is not accurate for now. We would need more time to optimize the weights to use and to filter non-ingredient words.

As an alternative option, we envision to clean the ingredient list by hand.

This algorithm does not take into account the statistical relevance of letters in the english language, but only alphabetical closeness. 

# 4. Cooking time study-case

In this part we would like to analyze the cooking time of the recipes to be able to classify which regions have the highest and lowest cooking time.

In [70]:
# Extract all timing from recipes
timing_df = allrecipes_desc_df['Description'].str.extractall(r'(\d+) minutes|(\d+) hour|hours')
timing_df.columns = ['minutes', 'hours']

#Replace Nan by 0 and switch to int type
timing_df = timing_df.fillna(0).astype(int)

#Change the hour into minutes and sum the two columns
timing_df['hours']=timing_df['hours']*60

#Sum the number of minutes to get the recipe time
timing_df['Recipe Time (min)'] = timing_df['minutes']+timing_df['hours']

timing_df.head()

minutes  hours  Recipe Time (min)
ID                               match                                   
b9705d990df6857f20756fc996a54b63 0            0    120                120
                                 1           15      0                 15
beed004e2a1772ba0db9da913f54122e 0            0    240                240
                                 1            5      0                  5
ee659a6a5e69834b60744cc3e103729e 0           12      0                 12

In [69]:
# Sum the total amount of time for each recipe
time_recipe = timing_df.groupby('ID').agg('sum')
time_recipe = time_recipe.drop(['minutes','hours'], axis=1)
time_recipe.head()

,Recipe Time (min)
ID,
00035a69b44a9dd1f88f2bb5faced261,90
000d31e632cab9e6902f05196354a007,660
0015417d2473d92a56da28883a27aff3,120
001cf1a5a0d1914f958cb2c823df6121,8
001f5efe07f4c72b4aaf846ec7616aba,13


In [ ]:
plt.hist(time_recipe, bins=100)
plt.show()

# 5. Visualization

In [ ]:
# Interactive plot of correlation between nutritive values 
def f(nutritive1, nutritive2):
    
    sns.set_context("notebook", font_scale=1.5)
    sns.scatterplot(recipes_df[nutritive1], recipes_df[nutritive2])
    plt.show()
    
# Interact
interact(f, nutritive1=['kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol'],
            nutritive2=['kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol']);

In the plot above we can see the correlation between the different nutritional values. 

For example, there are many recipes where high carbs and fats correspond to high caloric plates, but less so for high proteins. Also it would seem that fats and cholesterol are not as correlated as we would think.

Below is a plot that shows the correlation coefficient for pairs of nutritional values by region. 

In [ ]:
# Correlation between nutritional values shown per region
def f(region):
    sns.set_context("notebook", font_scale=1.5)
    
    corr = recipes_df[recipes_df['Region'] == region].corr()
    sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()
    
# Interact
interact(f, region=recipes_df.Region.unique().dropna());

Below is a plot that shows the statistics by nutritional value of recipes classified by region. The plot is automatically ordered by median, so we obtain the region with the highest median for that nutritional value.

In [ ]:
# Nutritional value statistics by regions
def f(nutritive):
    recipe_sorted = recipes_regions.sort_values([(nutritive, 'median')], ascending=False)

    sns.set_context("notebook", font_scale=1.5)
    sns.boxplot(recipes_df[nutritive], recipes_df['Region'], order=recipe_sorted.index)
    plt.show()
    
# Interact
interact(f, nutritive=['kcal', 'carb', 'fat', 'protein', 'sodium', 'cholesterol']);
# TODO for sodium put a xlim

We can see that the most calorical, fat and protein rich recipes belong to Malaysia, while the sodium intake is won by the korean recipes. The ones that have to be most careful about the cholesterol intake seem to be the French.

TODO outliers

In [ ]:
# Loading JSON of world map
world_json = json.load(open(DATA_FOLDER + 'world-countries.json'))
cont_json = json.load(open(DATA_FOLDER + 'continents.json'))

In [ ]:
# Creating a new map
m_world= folium.Map([30,0], tiles='cartodbpositron', zoom_start=2)
folium.GeoJson(world_json).add_to(m_world)
m_world

In [ ]:
# Creating a new map
m_continent= folium.Map([30,0], tiles='cartodbpositron', zoom_start=2)
folium.GeoJson(cont_json).add_to(m_continent)
m_continent

-> custom map: https://geojson-maps.ash.ms/